In [ ]:
# --- CELL 1: SETUP & DATA ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# 1. Đọc dữ liệu
df = pd.read_csv("Housing.csv")

# 2. Xử lý dữ liệu (Mấu chốt để không bị lỗi)
# Chuyển toàn bộ cột chữ (yes/no) thành số (1/0)
df_encoded = pd.get_dummies(df, drop_first=True)

print("Dữ liệu sau khi mã hóa:")
display(df_encoded.head())

# 3. Tách X, y
X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

# 4. Chia Train/Test
# .values để chuyển sang mảng Numpy ngay từ đầu -> Tính toán tay không bị lệch index
x_train, x_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

print(f"\nĐã chia xong: Train ({len(x_train)}), Test ({len(x_test)})")

Dữ liệu sau khi mã hóa:


,price,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,2,True,False,False,False,True,True,False,False
1,12250000,8960,4,4,4,3,True,False,False,False,True,False,False,False
2,12250000,9960,3,2,2,2,True,False,True,False,False,True,True,False
3,12215000,7500,4,2,2,3,True,False,True,False,True,True,False,False
4,11410000,7420,4,1,2,2,True,True,True,False,True,False,False,False



Đã chia xong: Train (436), Test (109)


In [6]:
print("Số lượng ô bị trống (NaN):")
print(df.isnull().sum())

Số lượng ô bị trống (NaN):
price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64


In [7]:
# --- CELL 2: HELPER FUNCTION ---
def train_weak_learner(x_train, y_train, x_test, weights):
    # Khởi tạo cây nông (max_depth=3)
    tree = DecisionTreeRegressor(max_depth=3, random_state=42)
    
    # Dạy cây với trọng số (sample_weight)
    tree.fit(x_train, y_train, sample_weight=weights)
    
    # Trả về cây và kết quả dự đoán train (để tính lỗi)
    pred_train = tree.predict(x_train)
    
    return tree, pred_train

In [8]:
# --- CELL 3: MAIN ADABOOST LOOP ---
epoche = 50
n_samples = len(x_train)
weights = np.ones(n_samples) / n_samples  # Ban đầu chia đều trọng số (1/N)

models = [] # Lưu danh sách các cây
alphas = [] # Lưu tiếng nói (trọng số) của từng cây

print(f"Bắt đầu chạy AdaBoost thủ công ({epoche} vòng)...")

for i in range(epoche):
    # A. Gọi hàm huấn luyện
    tree, pred_train = train_weak_learner(x_train, y_train, x_test, weights)
    
    # B. Tính Lỗi (Error) trên tập Train
    err = np.abs(y_train - pred_train)
    err /= err.max() # Chuẩn hóa lỗi về [0, 1]
    
    # C. Tính Alpha (Tiếng nói của cây)
    avg_err = np.sum(weights * err)
    beta = avg_err / (1 - avg_err + 1e-10)
    alpha = np.log(1 / (beta + 1e-10))
    
    # D. Cập nhật Trọng số (Update Weights)
    # Mẫu nào sai nhiều (err lớn) -> weights tăng
    weights *= (beta ** (1 - err))
    weights /= weights.sum() # Chuẩn hóa lại tổng = 1
    
    # Lưu lại mô hình và alpha
    models.append(tree)
    alphas.append(alpha)
    
print("Hoàn thành huấn luyện!")

Bắt đầu chạy AdaBoost thủ công (50 vòng)...
Hoàn thành huấn luyện!


In [9]:
# --- CELL 4: PREDICT & EVALUATE ---
# 1. Tổng hợp kết quả (Weighted Sum)
final_pred = np.zeros(len(x_test))
sum_alphas = sum(alphas)

# Lôi từng cây trong kho ra dự đoán
for tree, alpha in zip(models, alphas):
    final_pred += alpha * tree.predict(x_test)

# Chia cho tổng alpha để lấy trung bình
final_pred /= sum_alphas

# 2. Tính RMSE
rmse = np.sqrt(mean_squared_error(y_test, final_pred))

print("=== KẾT QUẢ CUỐI CÙNG ===")
print(f"RMSE Manual AdaBoost: {rmse:,.2f}")

=== KẾT QUẢ CUỐI CÙNG ===
RMSE Manual AdaBoost: 1,523,329.75
